In [69]:
CUDA_LAUNCH_BLOCKING="1"

In [70]:
import re
import math
import time
import scipy
import collections
import multiprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
# from datasets import load_dataset
from collections import defaultdict
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
# from torcheval.metrics.text import Perplexity

In [71]:
#TODO delete words with less than 5 times repete from vocab @done


In [72]:
val_file_path='NLP-01-2-HW1-Data/valid.txt'
mask_file_path='NLP-01-2-HW1-Data/mask.txt'
mask_gold_file_path='NLP-01-2-HW1-Data/mask_gold.txt'

incomplete_file_path='NLP-01-2-HW1-Data/incomplete.txt'

In [73]:
# mode='coding'
mode='training'
if mode =='training':
    corpus_file_path = 'NLP-01-2-HW1-Data/train.txt'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    num_epochs = 20
else:
    corpus_file_path = 'NLP-01-2-HW1-Data/test.txt'

    # device = 'cpu'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    num_epochs = 2

print(corpus_file_path)

NLP-01-2-HW1-Data/train.txt


In [74]:
available_workers = multiprocessing.cpu_count()
print(f' using {device} with {available_workers} cpus')

 using cuda with 16 cpus


In [75]:
def load_dataset(path):
    with open(path, 'r') as file:
        txt=file.read()
    return txt

In [76]:
def preprocess_txt(txt):
    '''Preprocess the text by removing any unwanted characters,
     converting all the text to lowercase'''
    # txt=" ".join(re.findall(r"[\u0600-\u06FF]+", txt)) # delete all non_ persion texts

    # remove any unwanted characters
    # txt=txt.replace('\u200c',' ')
    txt=txt.replace('\n',' ')
    # txt = re.sub("\d+" ," ", txt)# delete numbers

    # txt= re.sub(r'[^\w\s]','',txt)
    # convert all the english text to lowercase
    txt=re.sub(' +', ' ',txt)
    # txt=txt.lower()
    
    
    return txt

In [77]:
preprocess_txt('همچنین جمالت فایلهای txt.mask . ۶۷و  hjg txt.incomplete ')

'همچنین جمالت فایلهای txt.mask . ۶۷و hjg txt.incomplete '

In [78]:
def txt2word(txt):
    # split the text into individual words.
    words= txt.split(' ')
    # remove '_' in case it appears individually as a word
    if "_" in words:
        words.remove("_")
    # delete empty strings
    if "" in words:
        words.remove("")
        
    return words

In [79]:
def txt2sentence(txt):
    # split the text into individual sentence.
    sentences=re.split(';|\.|\n|\?|\؟',txt)
    # delete empty strings
    if '' in sentences:
        sentences.remove('')
    
    return sentences

In [80]:
def create_vocab(corpus_txt):
    # create voacab of corpus and delete the low frequent word from it
    words_term_frequency_train={}
    # UNK_symbol = "<UNK>"
    vocab=set()
    words_list = txt2word(corpus_txt)
    # create term frequency of the words
    for word in words_list:
        words_term_frequency_train[word] = words_term_frequency_train.get(word,0) + 1
    # create vocabulary and delete low frequency words
    for word in words_list:
        if words_term_frequency_train.get(word,0) >= 5:
            vocab.add(word)
    return list(vocab)

In [81]:
class CorpusDataset(Dataset):
    def __init__(self, corpus_file_path, vocab,n=2):
        
        self.n=n
        self.corpus =self.load_dataset(corpus_file_path)
        self.corpus=[self.preprocess_txt(sentence) for sentence in self.txt2sentence(self.corpus)]
        self.vocab = vocab
        self.vocab_size=len(vocab)+1
        self.clean_corpus()
        self.word_to_id_mappings=self.create_mapper()
        self.X,self.y=self.create_dataset()



    def create_mapper(self):
        word_to_id_mappings={'<UNK>':0}
        for idx, word in enumerate(self.vocab,1):
            word_to_id_mappings[word]=idx
        return word_to_id_mappings

    def word2idx(self,word):
        unknown_word_id = self.word_to_id_mappings['<UNK>']
        return self.word_to_id_mappings.get(word,unknown_word_id)

    def load_dataset(self,path):
        with open(path, 'r') as file:
            txt=file.read()
        return txt

    def preprocess_txt(self,txt):
        '''Preprocess the text by removing any unwanted characters,
        converting all the text to lowercase'''
        # delete all non_ persion texts
        txt=" ".join(re.findall(r"[\u0600-\u06FF]+", txt)) 
        # remove any unwanted characters
        txt=txt.replace('\u200c',' ')
        txt=txt.replace('\n',' ')
        txt = re.sub("\d+" ," ", txt)# delete numbers
        txt= re.sub(r'[^\w\s]','',txt)
        # convert all the english text to lowercase
        txt=re.sub(' +', ' ',txt)
        # convert all the english text to lowercase
        txt=txt.lower()  
        return txt

    def txt2sentence(self,txt):
        # split the text into individual sentence.
        sentences=re.split('|;|\.|\n|\?|\؟',txt)
        # delete empty strings
        sentences=list (filter(lambda sent : sent!= '', sentences))
        
        return sentences
        
    
    def clean_corpus(self):
        self.corpus=list (filter(lambda sentence : sentence!= '', self.corpus))


    def txt2word(self,txt):
        # split the text into individual words.
        words= txt.split(' ')
        # remove '_' in case it appears individually as a word
        if "_" in words:
            words.remove("_")
        # delete empty strings
        if "" in words:
            words.remove("")
            
        return words
    

    def create_dataset(self):
        X=[]
        y=[]
        for sentence in self.corpus:
            words_list=self.txt2word(sentence)
            for i in range(len(words_list)):
                if len(words_list)<= i+self.n:
                    # ignoring sentece less than contex size 
                    # (if n=3 less than 4 words )
                    break
                
                x_extract=[self.word2idx(words_list[i+j]) for j in range(self.n) ]
                y_extraxt=[self.word2idx(words_list[i+self.n])]
                # for n=3 x is[i,i+1,i+2]
                # and y is [i+3]
                
                X.append(x_extract)
                y.append(y_extraxt)
        
        return np.array(X),np.array(y)



    def __getitem__(self, idx):
        
        x_1 = torch.from_numpy(self.X[idx]).to(device)
        y_1 = torch.tensor(self.y[idx]).to(device)
        return x_1,y_1
        
    
    def __len__(self):
        return len(self.X)




In [82]:
# train_corpus=load_dataset(corpus_file_path)
# train_corpus=preprocess_txt(train_corpus)
# create_vocab(train_corpus)

In [83]:
class FFNNLanguageModel(nn.Module):
    def __init__(self,vocab_size, embedding_dim, hidden_dim,n=2):
        ''' output_dim=vocab_size
        n : N previous word that we want to predict based on them (input context_size)
        '''
        super().__init__()
        self.embedding_dim = embedding_dim
        self.embedding=nn.Embedding(vocab_size, embedding_dim)# embeding layer E(V*d)
        self.hidden_layer=nn.Linear(n*embedding_dim, hidden_dim)# hidden layer W(nd*d_h)
        self.output_layer=nn.Linear(hidden_dim, vocab_size,bias = False)# output layer U (d_h*V)
        self.n=n #input context_size
    def forward(self,x):
 
        # compute X : v*nd  if n=3 it will concatinate X1,X2,X3 for trigram model
        embedded = self.embedding(x).view((-1,self.n * self.embedding_dim))
        # compute  hidden layer
        hidden = F.relu(self.hidden_layer(embedded))
        # compute softmax of output
        output = F.softmax(self.output_layer(hidden), dim=1)
        # return log probabilities
        # BATCH_SIZE x len(vocab)
        return output

In [84]:
def get_accuracy_from_log_probs(log_probs, labels):
    probs = torch.exp(log_probs)
    predicted_label = torch.argmax(probs, dim=1)
    acc = (predicted_label == labels).float().mean()
    return acc

In [85]:
def plot_charts(train_losses,train_accu):
    # plot accuracy and loss chart
    fig = plt.figure()
    plt.plot(list(range(num_epochs)), train_losses, color='blue')
    plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
    plt.xlabel('epochs')
    plt.ylabel('negative log likelihood loss')
    plt.savefig("loss.png")
    plt.show()

    fig = plt.figure()
    plt.plot(list(range(num_epochs)), train_accu, color='blue')
    plt.legend(['Train accuracy'], loc='upper right')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.savefig("loss.png")
    plt.show()


In [86]:

def train_model(data_loader,model,criterion,optimizer,num_epochs):
    train_losses = []
    train_accu=[]
    for epoch in range(num_epochs):
        running_loss = 0
        st = time.time()
        for i, (inputs, labels) in enumerate(data_loader):
            optimizer.zero_grad()
            outputs = model(inputs)

            acc = get_accuracy_from_log_probs(outputs, labels)
            
            loss = criterion(outputs,labels.squeeze())

            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 1000 == 0: 
                print("Training Iteration {} of epoch {}  Loss: {}; Acc:{}; Time taken (s): {}".format(
                    i, epoch, loss.item(), acc, (time.time()-st)))
                st = time.time()
        train_losses.append(running_loss/len(data_loader))
        train_accu.append((100*acc.cpu().numpy()))
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/i:.4f},acc: {acc}')
    return model,train_losses,train_accu

# trigram NN model


In [87]:
context_size=3

## create vocab

In [88]:
train_corpus=load_dataset(corpus_file_path)
train_preprocess_corpus=preprocess_txt(train_corpus)
# train_vocab=create_vocab(train_corpus)
vocab= create_vocab(train_corpus)

In [89]:
val_corpus=load_dataset(val_file_path)
val_preprocess_corpus=preprocess_txt(train_corpus)

# set up model

In [90]:
embedding_dim = 200#len(train_dense_embed)
hidden_dim = 100
vocab_size = len(vocab)+1

# laod data and create trigram

In [91]:
train_corpus=load_dataset(corpus_file_path)
train_preprocess_corpus=preprocess_txt(train_corpus)
# train_vocab=create_vocab(train_corpus)
vocab= create_vocab(train_corpus)

In [92]:
# mask_dataset = CorpusDataset(mask_file_path, vocab, n=context_size)
# mask_data_loader = DataLoader(mask_dataset, batch_size=1, shuffle=False)

In [93]:
def create_mapper(vocab):
    word_to_id_mappings={'<UNK>':0}
    for idx, word in enumerate(vocab,1):
        word_to_id_mappings[word]=idx
    return word_to_id_mappings

def word2idx(word,word_to_id_mappings):
    unknown_word_id = word_to_id_mappings['<UNK>']
    return int(word_to_id_mappings.get(word,unknown_word_id))


In [94]:
def idx2word(id,vocab):
    if id==0:
        return '<UNK>'
    else:
        return list(vocab)[id+1]

In [95]:
word_to_id_mappings=create_mapper(vocab)

In [96]:
mask_data=[]
with open(mask_file_path) as file_in:
    for line in file_in:
        words=(re.split('\u200c| ',preprocess_txt(line.split('\t')[1])))
        for j in range(words.count('#')):
            if j==0:
                idx=0
            idx=words.index('#',idx+1)
            if idx <=3:
                trigram=['']+words[idx-2:idx]
                # print(trigram)
            else:
                trigram=(words[idx-3:idx])
            x_extract=np.array([[word2idx(trigram[i],word_to_id_mappings) \
                for i in range(3)]],dtype=np.int64)
            x_extract=torch.from_numpy(x_extract).to(device)
            mask_data.append([x_extract,trigram])
        

In [98]:
# x_extract=[word2idx(tri_model[i],word_to_id_mappings) for  in range(3)]
#             print(x_extract)

In [99]:
tri_model=torch.load('./FFLM_20_3gram.pth')

In [100]:
vocab.append('#')

In [101]:
tri_model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for inputs ,trigram in mask_data:
        print(trigram)
        output = tri_model(inputs)
        out=(output.cpu().numpy())
        if np.argmax(out)==0 and np.max(out)!=1:
            idx=out.argsort()[:,-2][0]
        else:
            idx=np.argmax(out)
        # print(idx)
        print(idx2word(idx,vocab))
        # break
        # print(inputs)

        

['فعال', 'از', 'نظر']
<UNK>
['در', 'سال', '۱۸۹۹']
<UNK>
['کسب', 'یک', 'عنوان']
<UNK>
['', 'بزرگترین', 'کلیسای']
<UNK>
['خان', 'پس', 'از']
<UNK>
['بر', 'دشمنان', 'خود']
<UNK>
['', 'عمومی', 'ترین']
<UNK>
['که', 'طی', 'مراحل']
<UNK>
['پیش', 'از', 'همه']
<UNK>


In [109]:
incomplete_data=[]
with open(incomplete_file_path) as file_in:
    for line in file_in:
        words=(re.split('\u200c| ',preprocess_txt(line.split('\t')[1])))
        idx=len(words)
        # if idx <=3:
        #     trigram=['']+words[idx-2:idx]
        #     # print(trigram)
        # else:
        trigram=(words[idx-4:idx-1])
        x_extract=np.array([[word2idx(trigram[i],word_to_id_mappings) \
            for i in range(3)]],dtype=np.int64)
        x_extract=torch.from_numpy(x_extract).to(device)
        incomplete_data.append([x_extract,trigram])

In [110]:
tri_model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for inputs ,trigram in incomplete_data:
        print(trigram)
        output = tri_model(inputs)
        out=(output.cpu().numpy())
        if np.argmax(out)==0 and np.max(out)!=1:
            idx=out.argsort()[:,-2][0]
        else:
            idx=np.argmax(out)
        # print(idx)
        print(idx2word(idx,vocab))
        # break
        # print(inputs)

['محمدعلی', 'شاه', 'با']
<UNK>
['خدمات', 'مالی', 'و']
<UNK>
['موسی', 'تورات', 'را']
<UNK>
['یک', 'لغت', 'نامه']
<UNK>
['در', 'ابتدا', 'شعبه']
<UNK>


''